In [2]:
import pandas as pd
import numpy as np

In [90]:
import re
import string

punctuation = string.punctuation

def process_string(text):
    text = text.translate(str.maketrans("", "", punctuation))
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()

In [91]:
import requests
from bs4 import BeautifulSoup

class WebScraper:
    def __init__(self, url) -> None:
        self.url = url
        self.soup = self.get_html_content()

    def get_html_content(self):
        try:
            # Make the HTTP request
            response = requests.get(self.url)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the HTML content
                soup = BeautifulSoup(response.content, 'html.parser')
                return soup
            else:
                return f"Failed to retrieve the page. Status code: {response.status_code}"
        except requests.exceptions.RequestException as e:
            return f"An error occurred: {e}"
    
    def get_text_by_classname(self, class_name="message-body js-selectToQuote"):
        texts = []
        try:
            elements = self.soup.find_all(class_=class_name)
            for element in elements:
                texts.append(process_string(element.get_text().replace('\n', ' ').replace('\t', ' ').replace('\xa0', '')))

            return texts

        except Exception as e:
            return f"An error occurred while fetching the elements: {e}"
    

In [92]:
scrapper = WebScraper("https://forum.adrenaline.com.br/threads/indicacoes-de-aquisicao-de-pc-novo-configuracao-completa-gamer-profissional-uso-casual.691749/")

In [93]:
scrapper.get_text_by_classname()

['tópico para concentrar mensagens de quem busca dicas e indicações para montar um pc novo',
 'pessoal uma dúvida sobre config to pensando em montar um pc com as especificações abaixo comprando peça por peça para dar uma economizada i5 11400f 16gb ram 2x8gb 3200 placa mãe h510mt m2 v20 rtx 3050 8gb fonte corsair 500w 80 plus white busco gastar no máximo 67k comprando uma peça de cada vez acompanhando as promoções etc vocês mudariam algo pc para uso no dia a dia trabalho homeoffice e jogos em full hd vi algumas pessoas falando mal da 11ª geração da intel pensei em talvez pegar um i3 12100f mas realmente não sei qual seria melhor edit pro meu trabalho não preciso de tanta coisa uso apenas o google chrome com várias abas abertas rsrs',
 'senhores deem uma luz aqui por favor estou querendo montar um pc para jogar em 4k e 60fps mas dependendo do orçamento final eu fico com o quadhd a ideia é montar um em torno do amd ryzen 5600x e da rtx3080 o que eu posso comprar de placa mãe cooler vram e

In [48]:

links = ['https://forum.adrenaline.com.br/threads/indicacoes-de-aquisicao-de-pc-novo-configuracao-completa-gamer-profissional-uso-casual.691749/']

In [96]:


comments = []
for i in range(1, 50):
    comments.extend(WebScraper(links[0] + f'page-{i}').get_text_by_classname())


In [98]:
df = pd.DataFrame(comments, columns=['comments'])

In [109]:
from textblob import TextBlob
import nltk

In [113]:
# nltk.download()

In [126]:
text = df['comments'][5]

blob = TextBlob(text)
blob.tags  # [('The', 'DT'), ('titular', 'JJ'),
#  ('threat', 'NN'), ('of', 'IN'), ...]

blob.noun_phrases  # WordList(['titular threat', 'blob',
#            'ultimate movie monster',
#            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print(sentence.sentiment)
# 0.060
# -0.341

Sentiment(polarity=0.35, subjectivity=0.55)


In [127]:
df['comments'][5]

'boa tarde pessoal tenho uma máquina frankenstein aqui i5 2310 12 gb ram genéricas rx 480 8 gb da msi fonte evga 650 w quero dar um up inicialmente na cpu queria opções para fazer essa mudança a placa de vídeo ta velhinha mas diante dos preços atuais estou me agarrando a ela mesma tenho preferência por processadores com vídeo integrado pois já tive a experiência de ter uma placa de vídeo queimada e ficar com o pc inutilizado por um tempo a finalidade do pc será para jogos mesmo porém me contento com um bom full hd com isso quais cpu mobo memoria posso estar pegando para ter uma máquina rodando os títulos atuais em 60 fps ps tenho disponível 3500 para atualizar'

In [128]:
df['sentiment'] = df['comments'].apply(lambda x: TextBlob(x).sentiment)

In [129]:
df

,comments,sentiment
0,tópico para concentrar mensagens de quem busca...,"(0.0, 0.0)"
1,pessoal uma dúvida sobre config to pensando em...,"(0.175, 0.275)"
2,senhores deem uma luz aqui por favor estou que...,"(0.1, 0.6)"
3,boa tarde pessoal ando meio desatualizado no q...,"(0.0, 0.0)"
4,retfield disse boa tarde pessoal ando meio des...,"(0.0, 1.0)"
...,...,...
1220,montar pc gamer para jogar diablo 4 ajudem pes...,"(0.375, 0.75)"
1221,krauseoasis disse montar pc gamer para jogar d...,"(0.16944444444444443, 0.5722222222222222)"
1222,a b350 ds3h v2 suporta a 5900x em stock com hi...,"(0.0, 0.0)"
1223,shinolo disse a b350 ds3h v2 suporta a 5900x e...,"(0.0, 0.0)"


In [130]:
df.to_csv('comments.csv', index=False)